# Importation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/corpus.csv')

# Séparation test et entrainement

In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv('../data/train.csv', index=False)
test_df.to_csv('../data/test.csv', index=False)
